In [1]:
import cv2
import  numpy as np
from time import sleep

#The below lines are to take input directly from webcam
# cap = cv2.VideoCapture(0)
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('output.avi',fourcc,5.0,(640,480))
#0 = webcam1
#1 = webcam2
file1 = 'video/video front1.mp4'
file2 = 'video/video front2.mp4'
file3 = 'video/back1.mp4'
file4 = 'video/back2.mp4'
file5 = 'video/emergency1.mp4'
file6 = 'video/emergency2.mp4'


# Function to count no of cars


In [2]:
def getCenter(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

In [3]:
def countCarsOnEachLane(filename,pos_line=650):
    min_width = 80 #min width of rectangle
    min_height = 80 #min height of rectangle
    # max_width = 250
    # max_height = 250
    offset = 6     #Allowed error per pixel  

    #pos_line = 650    #Count line position (default for back1 is 380 and back2 is 390 and 560 for front)

    delay = 60 #FPS vídeo

    detect = []
    cars = 0

    cap = cv2.VideoCapture(filename)
    subtract = cv2.bgsegm.createBackgroundSubtractorMOG()

    while True:
        ret , frame1 = cap.read()
        time = float(1 / delay)
        sleep(time) 
        grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (3, 3), 5)
        img_sub = subtract.apply(blur)
        dilate = cv2.dilate(img_sub, np.ones((5, 5)))
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        dilated = cv2.morphologyEx (dilate, cv2. MORPH_CLOSE , kernel)
        dilated = cv2.morphologyEx (dilated, cv2. MORPH_CLOSE , kernel)
        contour , h = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        cv2.line(frame1, (0, pos_line), (1200, pos_line), (255, 127, 0), 3) #frame,starting pos,ending pos,color,thickness
        for(i, c) in enumerate(contour):
            (x, y, w, h) = cv2.boundingRect(c)
            validate_contour = (w >= min_width) and (h >= min_height)
            #   removing the below line which also checks for max height and width  
            #  validate_contour= (((w >= min_width) and (h >= min_height)) and ((w<=max_width) and (h<=max_height)))
            if not validate_contour:
                continue

            cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)        
            center = getCenter(x, y, w, h)
            detect.append(center)
            cv2.circle(frame1, center, 4, (0, 0, 255), -1)

            for (x,y) in detect:
                if y<(pos_line + offset) and y > (pos_line - offset):
                    cars += 1
                    cv2.line(frame1, (25, pos_line), (1200, pos_line), (0, 127, 255), 3)  
                    detect.remove((x, y))
                    #print("cars detected : "+str(cars))        

        cv2.putText(frame1, "VEHICLE COUNT : " + str(cars), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Video Original" , frame1)
        cv2.imshow("Detected", dilated)

        if cv2.waitKey(1) == 27:   #key 27 is escape
            break

    cv2.destroyAllWindows()
    cap.release()
    return cars


# Function to detect emergency vehicle


In [4]:
def checkForEmergencyVehicle(filename):
    
    emergency_cascade = cv2.CascadeClassifier('cascade/firetruck.xml')
    #print(emergency_cascade.empty())
    cap = cv2.VideoCapture(filename)
    #cap = cv2.VideoCapture(0)			# For detection using web camera
    is_emergency_vehicle_present = 0
    emergency_vehicle_detected = 0
    while 1:
        ret, img = cap.read()
        #print(cv2.COLOR_BGR2GRAY)	
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #print(gray)

        emergency = emergency_cascade.detectMultiScale(gray, 1.11, 2)

        for (x, y, w, h) in emergency:
            cv2.rectangle(img, (x, y),(x + w, y + h),(255, 255, 0), 2)
            emergency_vehicle_detected += 1
            break

        if emergency_vehicle_detected == 5:
            #print("detected")
            is_emergency_vehicle_present = 1
            break

        cv2.imshow('Emergency Vehicle', img)
        k = cv2.waitKey(30) & 0xff
        if k == 27: #key 27 is escape key
            break

    cap.release()
    cv2.destroyAllWindows()
    return is_emergency_vehicle_present


# Status of lane function


In [5]:
#lane is a list with lane[0]->no of cars, lane[1]->emergency vehicle boolean,lane[2]->lane number

def statusOfLane(lane):
    
    print("Status of lane", lane[2])
    print ("No. of cars on lane", lane[2],": ", lane[0] )
    
    if lane[1] == 1:
        print("Emergency vehicle detected on lane", lane[2])
        
    else:
        print("Emergency vehicle not detected on lane", lane[2])


# Green Signal Check function


In [6]:
def greenSignalCheck(lane1, lane2, lane3, lane4):
    
    #BY default we have set green signal at lane 1
    greenSignal = lane1
    
    if lane1[1] == 1:
        #print("Green signal on lane",lane1[2])
        greenSignal = lane1
        return greenSignal
    
    if lane2[1] == 1:
        greenSignal = lane2
        return greenSignal
    
    if lane3[1] == 1:
        greenSignal=lane3
        return greenSignal
    
    if lane4[1] == 1:
        greenSignal=lane4
        return greenSignal
    
    
    if greenSignal[0] < lane2[0]:
        greenSignal = lane2
        
    if greenSignal[0] < lane3[0]:
        greenSignal = lane3
        
    if greenSignal[0] < lane4[0]:
        greenSignal = lane4
        
    return greenSignal 


# Applying functions


In [7]:
count1 = countCarsOnEachLane(file1, 550)
emergency1 = checkForEmergencyVehicle(file1)

lane1 = [count1, emergency1, '1']
statusOfLane(lane1)


Status of lane 1
No. of cars on lane 1 :  12
Emergency vehicle not detected on lane 1


In [8]:
count2 = countCarsOnEachLane(file2, 560)
emergency2 = checkForEmergencyVehicle(file2)

lane2 = [count2, emergency2, '2']
statusOfLane(lane2)

Status of lane 2
No. of cars on lane 2 :  12
Emergency vehicle not detected on lane 2


In [9]:
count3 = countCarsOnEachLane(file3, 380)
emergency3 = checkForEmergencyVehicle(file3)

lane3 = [count3, emergency3, '3']
statusOfLane(lane3)

Status of lane 3
No. of cars on lane 3 :  1
Emergency vehicle not detected on lane 3


In [10]:
#if you want an extra lane without emergency vehicles then run this
# count4 = countCarsOnEachLane(file4, 390)
# emergency4 = checkForEmergencyVehicle(file4)

# lane4 = [count4, emergency4, '4']
# statusOfLane(lane4)

In [11]:
#can use both file 5&6 for emergency vehicle detection
emergency4 = checkForEmergencyVehicle(file6)
count4 = 1
lane4 = [count4, emergency4, '4']
statusOfLane(lane4)

Status of lane 4
No. of cars on lane 4 :  2
Emergency vehicle detected on lane 4


In [12]:
print("Green Signal on lane", greenSignalCheck(lane1, lane2, lane3, lane4)[2])

Green Signal on lane 4



# Custom inputs


In [13]:
count1 = 10
count2 = 13
count3 = 11
count4 = 10
emergency1 = 0
emergency2 = 0
emergency3 = 0
emergency4 = 0

In [14]:
lane1 = [count1, emergency1, '1']
lane2 = [count2, emergency2, '2']
lane3 = [count3, emergency3, '3']
lane4 = [count4, emergency4, '4']

In [15]:
statusOfLane(lane1)
statusOfLane(lane2)
statusOfLane(lane3)
statusOfLane(lane4)

Status of lane 1
No. of cars on lane 1 :  10
Emergency vehicle not detected on lane 1
Status of lane 2
No. of cars on lane 2 :  13
Emergency vehicle not detected on lane 2
Status of lane 3
No. of cars on lane 3 :  11
Emergency vehicle not detected on lane 3
Status of lane 4
No. of cars on lane 4 :  10
Emergency vehicle not detected on lane 4


In [16]:
print("Green Signal on lane", greenSignalCheck(lane1, lane2, lane3, lane4)[2])

Green Signal on lane 2
